In [1]:
from tgb.linkproppred.dataset_pyg import PyGLinkPropPredDataset

In [2]:
name = 'tgbl-flight'

In [ ]:
dataset = PyGLinkPropPredDataset(name=name, root="datasets")

In [ ]:
import numpy as np

data = dataset.get_TemporalData()
data.t.shape
np.unique(data.t).shape, data.t.shape

In [ ]:
dataset = PyGLinkPropPredDataset(name=name, root="datasets")
train_mask = dataset.train_mask
val_mask = dataset.val_mask
test_mask = dataset.test_mask
data = dataset.get_TemporalData()

train_data = data[train_mask]
val_data = data[val_mask]
test_data = data[test_mask]

In [47]:
dataset.eval_metric
test_ns = dataset.load_test_ns()

In [ ]:
dataset.get_TemporalData()

In [ ]:
import torch

torch.min(train_data.t), torch.max(train_data.t), \
    torch.min(val_data.t), torch.max(val_data.t), \
    torch.min(test_data.t), torch.max(test_data.t)

In [ ]:
torch.unique(data.t)

In [ ]:
from StarDataset import StarDataset

summary_writer = 'TGN_new_wout_fish'
csv_name = 'tgn_wout_fish'
event_path = '/data/mala711/GNNthesis/data/Starboard/events.parquet'
data_path = '/data/mala711/GNNthesis/data/Starboard/vessels.csv'
batch_size = 5000
val_ratio = 0.15
test_ratio = 0.15
memory_dim = time_dim = embedding_dim = 100
epochs = 20
lr = 0.0001
l1_reg = 0
verbose = True
self_loop = False
with_fish = False
date_range = 'all'

sb_data = StarDataset(event_path, data_path, date_range, with_fish, self_loop)

data = sb_data.get_data()
ind_map = sb_data.get_ind_map()
rev_ind_map = sb_data.get_rev_ind_map()
rev_event_dict = sb_data.get_rev_event_dict()



In [ ]:
val_ratio = 0.10
test_ratio = 0.10

train_data, val_data, test_data = data.train_val_test_split(
    val_ratio=val_ratio, test_ratio=val_ratio)

torch.min(train_data.t), torch.max(train_data.t), \
    torch.min(val_data.t), torch.max(val_data.t), \
    torch.min(test_data.t), torch.max(test_data.t)

In [ ]:
from tgb.linkproppred.negative_generator import NegativeEdgeGenerator

num_neg_e_per_pos = 20
neg_sample_strategy = "hist_rnd" #"rnd"
rnd_seed = 42

# Ensure to only sample actual destination nodes as negatives.
min_dst_idx, max_dst_idx = int(data.dst.min()), int(data.dst.max())

# After successfully loading the dataset...
if neg_sample_strategy == "hist_rnd":
    historical_data = train_data
else:
    historical_data = None

neg_sampler = NegativeEdgeGenerator(
    dataset_name=name,
    first_dst_id=min_dst_idx,
    last_dst_id=max_dst_idx,
    num_neg_e=num_neg_e_per_pos,
    strategy=neg_sample_strategy,
    rnd_seed=rnd_seed,
    historical_data=historical_data,
)

# generate evaluation set
partial_path = "/data/mala711/GNNthesis/Baselines"
# generate validation negative edge set
split_mode = "val"
print(
    f"INFO: Start generating negative samples: {split_mode} --- {neg_sample_strategy}"
)
neg_sampler.generate_negative_samples(
    data=val_data, split_mode=split_mode, partial_path=partial_path
)
print(
    f"INFO: End of negative samples generation"
)

# generate test negative edge set
split_mode = "test"
print(
    f"INFO: Start generating negative samples: {split_mode} --- {neg_sample_strategy}"
)
neg_sampler.generate_negative_samples(
    data=test_data, split_mode=split_mode, partial_path=partial_path
)
print(
    f"INFO: End of negative samples generation"
)

In [63]:
from tgb.linkproppred.negative_sampler import NegativeEdgeSampler

negative_sampler = NegativeEdgeSampler(
    dataset_name = 'starboard',
    first_dst_id = min_dst_idx,
    last_dst_id = max_dst_idx,
)

In [65]:
negative_sampler.load_eval_set('/data/mala711/GNNthesis/Baselines/starboard_val_ns.pkl', 'val')

# TEST TGN

In [1]:
"""
Dynamic Link Prediction with a TGN model with Early Stopping
Reference: 
    - https://github.com/pyg-team/pytorch_geometric/blob/master/examples/tgn.py

command for an example run:
    python examples/linkproppred/tgbl-flight/tgn.py --data "tgbl-flight" --num_run 1 --seed 1
"""

import math
import timeit

import os
import os.path as osp
from pathlib import Path
import numpy as np
import tqdm
from StarDataset import StarDataset

import torch
from sklearn.metrics import average_precision_score, roc_auc_score
from torch.nn import Linear

from torch_geometric.datasets import JODIEDataset
from torch_geometric.loader import TemporalDataLoader

from torch_geometric.nn import TransformerConv

# internal imports
from tgb.utils.utils import get_args, set_random_seed, save_results
from tgb.linkproppred.evaluate import Evaluator
from modules.decoder import LinkPredictor
from modules.emb_module import GraphAttentionEmbedding

from modules.early_stopping import  EarlyStopMonitor

from torch_geometric.nn.models.tgn import (
    IdentityMessage,
    LastAggregator,
    LastNeighborLoader,
)
from torch_geometric.nn import TGNMemory
from tgb.linkproppred.dataset_pyg import PyGLinkPropPredDataset
from tgb.linkproppred.negative_sampler import NegativeEdgeSampler

In [2]:
# ==========
# ==========
# ==========


# Start...
start_overall = timeit.default_timer()
DATA = "starboard"

LR = 1e-4
BATCH_SIZE = 200
K_VALUE = 10 
NUM_EPOCH = 30
SEED = 1123
MEM_DIM = 128
TIME_DIM = 128
EMB_DIM = 128
TOLERANCE = 1e-6
PATIENCE = 5
NUM_RUNS = 5
SAVE = False
NUM_NEIGHBORS = 10


MODEL_NAME = 'TGN'
# ==========

# set the device
gpu_num = 5
device = torch.device(f"cuda:{gpu_num}" if torch.cuda.is_available() else "cpu")

# data loading
# dataset = PyGLinkPropPredDataset(name=DATA, root="datasets")
# train_mask = dataset.train_mask
# val_mask = dataset.val_mask
# test_mask = dataset.test_mask
# data = dataset.get_TemporalData()
# data = data.to(device)
metric = 'mrr'

# Starboard data
event_path = '/data/mala711/GNNthesis/data/Starboard/events.parquet'
data_path = '/data/mala711/GNNthesis/data/Starboard/vessels.csv'
val_file = '/data/mala711/GNNthesis/Baselines/starboard_val_ns.pkl'
test_file = '/data/mala711/GNNthesis/Baselines/starboard_test_ns.pkl'

val_ratio = 0.10
test_ratio = 0.10
self_loop = False
with_fish = False
date_range = ['2010-11-15', '2010-11-25']

sb_data = StarDataset(event_path, data_path, date_range, with_fish, self_loop)

data = sb_data.get_data().to(device)
ind_map = sb_data.get_ind_map()
rev_ind_map = sb_data.get_rev_ind_map()
rev_event_dict = sb_data.get_rev_event_dict()

# train_data = data[train_mask]
# val_data = data[val_mask]
# test_data = data[test_mask]

train_data, val_data, test_data = data.train_val_test_split(val_ratio=val_ratio, test_ratio=val_ratio)

train_loader = TemporalDataLoader(train_data, batch_size=BATCH_SIZE)
val_loader = TemporalDataLoader(val_data, batch_size=BATCH_SIZE)
test_loader = TemporalDataLoader(test_data, batch_size=BATCH_SIZE)

# Ensure to only sample actual destination nodes as negatives.
min_dst_idx, max_dst_idx = int(data.dst.min()), int(data.dst.max())

negative_sampler = NegativeEdgeSampler(
    dataset_name = DATA,
    first_dst_id = min_dst_idx,
    last_dst_id = max_dst_idx,
)

100%|██████████| 10330/10330 [00:00<00:00, 15382.50it/s]


In [10]:
data.t

tensor([0, 0, 0,  ..., 9, 9, 9], device='cuda:5')

In [11]:
for i in train_loader:
    batch = i
    break

In [18]:
batch.dst

tensor([1520, 1187, 1453, 1434, 1515, 1515,  111, 1547, 1515,  844, 2272, 1769,
        3192,  788, 1567, 1503, 3210, 2205,  788,  468, 1432, 1454, 1447,  510,
        1833, 1415,   35,  104, 1958, 1507,   97, 1820, 1473, 1473, 1486,  917,
        1538, 1563, 1507,    5,  785, 1467,   50, 1415, 1494, 1434, 3185,   46,
         991,  388, 1520,  169, 2453, 2323, 1507,   56, 1226, 3113, 3205, 1409,
        1112, 2313,   48, 1461,  773,  778, 1399,   15,  156, 1084, 1854, 2146,
        1391, 2064, 1487, 1420,  202, 1891, 1662, 1256, 1956, 1967, 1420, 1449,
        1390,  185, 1409,   32, 1494, 1102, 1520, 1126, 2828, 2821, 1126, 1126,
        1453, 1040,   60, 1226, 1820,  416, 1399, 1861,   15, 1515,  839, 1999,
        1769, 3192, 1515, 1833, 1396, 1449, 1515,  111, 1416, 1099, 1399, 1515,
        1102,  111,  167,  111, 1948, 2149, 1428, 3200, 1850, 1653, 1432, 2033,
        1677, 1416,   50, 1396, 1416, 1538,  163, 1487, 1416,  158,  839, 1409,
         338, 2651, 3205,  708, 3113,  1

In [8]:
# data loading
flight = PyGLinkPropPredDataset(name='tgbl-flight', root="datasets")
train_mask = flight.train_mask
val_mask = flight.val_mask
test_mask = flight.test_mask
flight_data = flight.get_TemporalData()
flight_data = flight_data.to(device)
flight_metric = flight.eval_metric

flight_train_data = flight_data[train_mask]
flight_val_data = flight_data[val_mask]
flight_test_data = flight_data[test_mask]

f_train_loader = TemporalDataLoader(flight_train_data, batch_size=BATCH_SIZE)
f_val_loader = TemporalDataLoader(flight_val_data, batch_size=BATCH_SIZE)
f_test_loader = TemporalDataLoader(flight_test_data, batch_size=BATCH_SIZE)

# Ensure to only sample actual destination nodes as negatives.
f_min_dst_idx, f_max_dst_idx = int(flight_data.dst.min()), int(flight_data.dst.max())

raw file found, skipping download
Dataset directory is  /data/mala711/miniconda3/envs/gnnthesis/lib/python3.10/site-packages/tgb/datasets/tgbl_flight
loading processed file


In [10]:
for i in f_train_loader:
    f_batch = i
    break

In [ ]:
data.y

In [ ]:
flight_data.y

In [19]:
neighbor_loader = LastNeighborLoader(data.num_nodes, size=NUM_NEIGHBORS, device=device)

# define the model end-to-end
memory = TGNMemory(
    data.num_nodes,
    data.msg.size(-1),
    MEM_DIM,
    TIME_DIM,
    message_module=IdentityMessage(data.msg.size(-1), MEM_DIM, TIME_DIM),
    aggregator_module=LastAggregator(),
).to(device)

gnn = GraphAttentionEmbedding(
    in_channels=MEM_DIM,
    out_channels=EMB_DIM,
    msg_dim=data.msg.size(-1),
    time_enc=memory.time_enc,
).to(device)

link_pred = LinkPredictor(in_channels=EMB_DIM).to(device)

model = {'memory': memory,
        'gnn': gnn,
        'link_pred': link_pred}

optimizer = torch.optim.Adam(
    set(model['memory'].parameters()) | set(model['gnn'].parameters()) | set(model['link_pred'].parameters()),
    lr=LR,
)
criterion = torch.nn.BCEWithLogitsLoss()

# Helper vector to map global node indices to local ones.
assoc = torch.empty(data.num_nodes, dtype=torch.long, device=device)


In [21]:
model['memory'].train()
model['gnn'].train()
model['link_pred'].train()

model['memory'].reset_state()  # Start with a fresh memory.
neighbor_loader.reset_state()  # Start with an empty graph.

total_loss = 0

prog_bar = tqdm.tqdm(range(len(train_loader)))
batch = batch.to(device)
optimizer.zero_grad()

src, pos_dst, t, msg = batch.src, batch.dst, batch.t, batch.msg

# Sample negative destination nodes.
neg_dst = torch.randint(
    min_dst_idx,
    max_dst_idx + 1,
    (src.size(0),),
    dtype=torch.long,
    device=device,
)

n_id = torch.cat([src, pos_dst, neg_dst]).unique()
n_id, edge_index, e_id = neighbor_loader(n_id)
assoc[n_id] = torch.arange(n_id.size(0), device=device)

# Get updated memory of all nodes involved in the computation.
z, last_update = model['memory'](n_id)
z = model['gnn'](
    z,
    last_update,
    edge_index,
    data.t[e_id].to(device),
    data.msg[e_id].to(device),
)

pos_out = model['link_pred'](z[assoc[src]], z[assoc[pos_dst]])
neg_out = model['link_pred'](z[assoc[src]], z[assoc[neg_dst]])

loss = criterion(pos_out, torch.ones_like(pos_out))
loss += criterion(neg_out, torch.zeros_like(neg_out))

# Update memory and neighbor loader with ground-truth state.
model['memory'].update_state(src, pos_dst, t, msg)
neighbor_loader.insert(src, pos_dst)

loss.backward()
optimizer.step()
model['memory'].detach()
total_loss += float(loss) * batch.num_events
prog_bar.update(1)

  2%|▏         | 1/42 [00:05<03:45,  5.49s/it]


In [ ]:
neighbor_loader(n_id)[2].shape

In [ ]:
batch.n_id

In [ ]:
torch.cuda.empty_cache()

In [22]:
# ==========
# ========== Define helper function...
# ==========

def train():
    r"""
    Training procedure for TGN model
    This function uses some objects that are globally defined in the current scrips 

    Parameters:
        None
    Returns:
        None
            
    """

    model['memory'].train()
    model['gnn'].train()
    model['link_pred'].train()

    model['memory'].reset_state()  # Start with a fresh memory.
    neighbor_loader.reset_state()  # Start with an empty graph.

    total_loss = 0

    prog_bar = tqdm.tqdm(range(len(train_loader)))
    for batch in train_loader:
        batch = batch.to(device)
        optimizer.zero_grad()

        src, pos_dst, t, msg = batch.src, batch.dst, batch.t, batch.msg

        # Sample negative destination nodes.
        neg_dst = torch.randint(
            min_dst_idx,
            max_dst_idx + 1,
            (src.size(0),),
            dtype=torch.long,
            device=device,
        )

        n_id, edge_index, e_id = neighbor_loader(batch.n_id)
        assoc[n_id] = torch.arange(n_id.size(0), device=device)

        # Get updated memory of all nodes involved in the computation.
        z, last_update = model['memory'](n_id)
        z = model['gnn'](
            z,
            last_update,
            edge_index,
            data.t[e_id].to(device),
            data.msg[e_id].to(device),
        )

        pos_out = model['link_pred'](z[assoc[src]], z[assoc[pos_dst]])
        neg_out = model['link_pred'](z[assoc[src]], z[assoc[neg_dst]])

        loss = criterion(pos_out, torch.ones_like(pos_out))
        loss += criterion(neg_out, torch.zeros_like(neg_out))

        # Update memory and neighbor loader with ground-truth state.
        model['memory'].update_state(src, pos_dst, t, msg)
        neighbor_loader.insert(src, pos_dst)

        loss.backward()
        optimizer.step()
        model['memory'].detach()
        total_loss += float(loss) * batch.num_events
        prog_bar.update(1)
    prog_bar.close()

    return total_loss / train_data.num_events


@torch.no_grad()
def test(loader, neg_sampler, split_mode):
    r"""
    Evaluated the dynamic link prediction
    Evaluation happens as 'one vs. many', meaning that each positive edge is evaluated against many negative edges

    Parameters:
        loader: an object containing positive attributes of the positive edges of the evaluation set
        neg_sampler: an object that gives the negative edges corresponding to each positive edge
        split_mode: specifies whether it is the 'validation' or 'test' set to correctly load the negatives
    Returns:
        perf_metric: the result of the performance evaluation
    """
    model['memory'].eval()
    model['gnn'].eval()
    model['link_pred'].eval()

    perf_list = []

    prog_bar = tqdm.tqdm(range(len(loader)))
    for pos_batch in loader:
        pos_src, pos_dst, pos_t, pos_msg = (
            pos_batch.src,
            pos_batch.dst,
            pos_batch.t,
            pos_batch.msg,
        )

        neg_batch_list = neg_sampler.query_batch(pos_src, pos_dst, pos_t, split_mode=split_mode)

        for idx, neg_batch in enumerate(neg_batch_list):
            src = torch.full((1 + len(neg_batch),), pos_src[idx], device=device)
            dst = torch.tensor(
                np.concatenate(
                    ([np.array([pos_dst.cpu().numpy()[idx]]), np.array(neg_batch)]),
                    axis=0,
                ),
                device=device,
            )

            n_id, edge_index, e_id = neighbor_loader(batch.n_id)
            assoc[n_id] = torch.arange(n_id.size(0), device=device)

            # Get updated memory of all nodes involved in the computation.
            z, last_update = model['memory'](n_id)
            z = model['gnn'](
                z,
                last_update,
                edge_index,
                data.t[e_id].to(device),
                data.msg[e_id].to(device),
            )

            y_pred = model['link_pred'](z[assoc[src]], z[assoc[dst]])

            # compute MRR
            input_dict = {
                "y_pred_pos": np.array([y_pred[0, :].squeeze(dim=-1).cpu()]),
                "y_pred_neg": np.array(y_pred[1:, :].squeeze(dim=-1).cpu()),
                "eval_metric": [metric],
            }
            perf_list.append(evaluator.eval(input_dict)[metric])

        # Update memory and neighbor loader with ground-truth state.
        model['memory'].update_state(pos_src, pos_dst, pos_t, pos_msg)
        neighbor_loader.insert(pos_src, pos_dst)
    
        prog_bar.update(1)
    prog_bar.close()

    perf_metrics = float(torch.tensor(perf_list).mean())

    return perf_metrics


print("==========================================================")
print(f"=================*** {MODEL_NAME}: LinkPropPred: {DATA} ***=============")
print("==========================================================")

evaluator = Evaluator(name=DATA)
neg_sampler = negative_sampler

# for saving the results...

if SAVE:
    results_path = f'/data/mala711/GNNthesis/Baselines/saved_results'
    if not osp.exists(results_path):
        os.mkdir(results_path)
        print('INFO: Create directory {}'.format(results_path))
    Path(results_path).mkdir(parents=True, exist_ok=True)
    results_filename = f'{results_path}/{MODEL_NAME}_{DATA}_results.json'

for run_idx in range(NUM_RUNS):
    print('-------------------------------------------------------------------------------')
    print(f"INFO: >>>>> Run: {run_idx} <<<<<")
    start_run = timeit.default_timer()

    # set the seed for deterministic results...
    # torch.manual_seed(run_idx + SEED)
    # set_random_seed(run_idx + SEED)

    # neighborhood sampler
    neighbor_loader = LastNeighborLoader(data.num_nodes, size=NUM_NEIGHBORS, device=device)

    # define the model end-to-end
    memory = TGNMemory(
        data.num_nodes,
        data.msg.size(-1),
        MEM_DIM,
        TIME_DIM,
        message_module=IdentityMessage(data.msg.size(-1), MEM_DIM, TIME_DIM),
        aggregator_module=LastAggregator(),
    ).to(device)

    gnn = GraphAttentionEmbedding(
        in_channels=MEM_DIM,
        out_channels=EMB_DIM,
        msg_dim=data.msg.size(-1),
        time_enc=memory.time_enc,
    ).to(device)

    link_pred = LinkPredictor(in_channels=EMB_DIM).to(device)

    model = {'memory': memory,
            'gnn': gnn,
            'link_pred': link_pred}

    optimizer = torch.optim.Adam(
        set(model['memory'].parameters()) | set(model['gnn'].parameters()) | set(model['link_pred'].parameters()),
        lr=LR,
    )
    criterion = torch.nn.BCEWithLogitsLoss()

    # Helper vector to map global node indices to local ones.
    assoc = torch.empty(data.num_nodes, dtype=torch.long, device=device)

    # define an early stopper
    if SAVE:
        save_model_dir = f'{osp.dirname(osp.abspath(__file__))}/saved_models/'
        save_model_id = f'{MODEL_NAME}_{DATA}_{SEED}_{run_idx}'
        early_stopper = EarlyStopMonitor(save_model_dir=save_model_dir, save_model_id=save_model_id, 
                                        tolerance=TOLERANCE, patience=PATIENCE)

    # ==================================================== Train & Validation
    # loading the validation negative samples
    neg_sampler.load_eval_set(val_file, 'val')

    val_perf_list = []
    train_times_l, val_times_l = [], []
    free_mem_l, total_mem_l, used_mem_l = [], [], []
    start_train_val = timeit.default_timer()
    for epoch in range(1, NUM_EPOCH + 1):
        # training
        start_epoch_train = timeit.default_timer()
        print(f'Training Epoch: {epoch:02d}')
        loss = train()
        end_epoch_train = timeit.default_timer()
        print(
            f"Loss: {loss:.4f}, Training elapsed Time (s): {end_epoch_train - start_epoch_train: .4f}"
        )
        # checking GPU memory usage
        free_mem, used_mem, total_mem = 0, 0, 0
        if torch.cuda.is_available():
            print("DEBUG: device: {}".format(torch.cuda.get_device_name(0)))
            free_mem, total_mem = torch.cuda.mem_get_info()
            used_mem = total_mem - free_mem
            print("------------Epoch {}: GPU memory usage-----------".format(epoch))
            print("Free memory: {}".format(free_mem))
            print("Total available memory: {}".format(total_mem))
            print("Used memory: {}".format(used_mem))
            print("--------------------------------------------")
        
        train_times_l.append(end_epoch_train - start_epoch_train)
        free_mem_l.append(float((free_mem*1.0)/2**30))  # in GB
        used_mem_l.append(float((used_mem*1.0)/2**30))  # in GB
        total_mem_l.append(float((total_mem*1.0)/2**30))  # in GB

        # validation
        start_val = timeit.default_timer()
        perf_metric_val = test(val_loader, neg_sampler, split_mode="val")
        end_val = timeit.default_timer()
        print(f"\tValidation {metric}: {perf_metric_val: .4f}")
        print(f"\tValidation: Elapsed time (s): {end_val - start_val: .4f}")
        val_perf_list.append(perf_metric_val)
        val_times_l.append(end_val - start_val)

        # check for early stopping
        if early_stopper.step_check(perf_metric_val, model):
            break

    train_val_time = timeit.default_timer() - start_train_val
    print(f"Train & Validation: Elapsed Time (s): {train_val_time: .4f}")

    # ==================================================== Test
    # first, load the best model
    early_stopper.load_checkpoint(model)

    # loading the test negative samples
    neg_sampler.load_eval_set(test_file, 'test')

    # final testing
    start_test = timeit.default_timer()
    perf_metric_test = test(test_loader, neg_sampler, split_mode="test")

    print(f"INFO: Test: Evaluation Setting: >>> ONE-VS-MANY <<< ")
    print(f"\tTest: {metric}: {perf_metric_test: .4f}")
    test_time = timeit.default_timer() - start_test
    print(f"\tTest: Elapsed Time (s): {test_time: .4f}")

    if SAVE:
        save_results({'data': DATA,
                    'model': MODEL_NAME,
                    'run': run_idx,
                    'seed': SEED,
                    'train_times': train_times_l,
                    'free_mem': free_mem_l,
                    'total_mem': total_mem_l,
                    'used_mem': used_mem_l,
                    'max_used_mem': max(used_mem_l),
                    'val_times': val_times_l,
                    f'val {metric}': val_perf_list,
                    f'test {metric}': perf_metric_test,
                    'test_time': test_time,
                    'train_val_total_time': np.sum(np.array(train_times_l)) + np.sum(np.array(val_times_l)),
                    }, 
        results_filename)

    print(f"INFO: >>>>> Run: {run_idx}, elapsed time: {timeit.default_timer() - start_run: .4f} <<<<<")
    print('-------------------------------------------------------------------------------')

print(f"Overall Elapsed Time (s): {timeit.default_timer() - start_overall: .4f}")
print("==============================================================")

=================*** TGN: LinkPropPred: starboard ***=============
-------------------------------------------------------------------------------
INFO: >>>>> Run: 0 <<<<<
Training Epoch: 01


  0%|          | 0/42 [00:00<?, ?it/s]

RuntimeError: CUDA error: CUBLAS_STATUS_EXECUTION_FAILED when calling `cublasSgemm( handle, opa, opb, m, n, k, &alpha, a, lda, b, ldb, &beta, c, ldc)`